In [1]:
import cifar10
import matplotlib.pyplot as plt

In [ ]:
cifar10.data_path = "data/CIFAR-10/"

In [ ]:
cifar10.maybe_download_and_extract()

In [ ]:
class_names = cifar10.load_class_names()
class_names

In [ ]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import ensemble

In [ ]:
images_train.shape

In [ ]:
fig=plt.figure(figsize=(8,8))
for i in range(64):
    ax=fig.add_subplot(8,8,i+1)
    ax.imshow(images_train[i],cmap=plt.cm.bone)
plt.show()

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
data_train = images_train.reshape(50000,3072)
data_test = images_test.reshape(10000,3072)

In [ ]:
from sklearn.decomposition import PCA
pca=PCA()
pca.fit(data_train)

In [ ]:
pca.components_.shape

In [ ]:
k=0
total=sum(pca.explained_variance_)
currentSum=0
while currentSum/total<0.95:
    currentSum+=pca.explained_variance_[k]
    k=k+1

In [ ]:
pca_t=PCA(n_components=k,whiten=True)

In [ ]:
x_transform = pca_t.fit_transform(data_train)
x_transform.shape

In [ ]:
x_approx = pca_t.inverse_transform(x_transform)
x_approx = x_approx.reshape((50000, 32, 32, 3))
x_approx.shape

In [ ]:
print(pca_t.components_.shape)
eigenv = pca_t.components_.reshape(k,32,32,3)

In [ ]:
x_train_pca = x_transform
x_test_pca = pca_t.transform(data_test)

In [ ]:
clf_RF = RandomForestClassifier()
grid_RF ={
    'n_estimators':[5,10,15,20],
    'max_depth':[5,10,20,30]
}
grid_search_RF = GridSearchCV(clf_RF, grid_RF)
grid_search_RF.fit(x_train_pca, cls_train)
grid_search_RF.best_estimator_
grid_search_RF.best_score_

In [ ]:
clf_RF = RandomForestClassifier()
grid_RF ={
    'n_estimators':[5,10,20,30],
    'max_depth':[10,20,30,35]
}
grid_search_RF = GridSearchCV(clf_RF, grid_RF)
grid_search_RF.fit(x_train_pca, cls_train)
grid_search_RF.best_estimator_
grid_search_RF.best_score_

In [ ]:
clf_SVC = svm.SVC()
grid_SVC = {'C' : [1e2, 1e3, 5e3, 1e4],
       'gamma' : [1e-3, 5e-4, 1e-4]}
grid_search_SVC = GridSearchCV(clf_SVC, grid_SVC,cv=model_selection.KFold(n_splits=10,shuffle=True,random_state=1))
grid_search_SVC.fit(x_train_pca,cls_train)
grid_search_SVC.best_score_

In [ ]:
clf_DT = DecisionTreeClassifier(criterion="entropy",splitter="best")
grid_DT = {'min_samples_split':[2,5,10,15,20,25,30],'max_depth':[10,15,20,25,30,35],
           'min_impurity_decrease':[10**-i for i in range(2, 5)]}
grid_search_DT = GridSearchCV(clf_DT, grid_DT,cv=model_selection.KFold(n_splits=10,shuffle=True,random_state=1))
grid_search_DT.fit(x_train_pca,cls_train)
grid_search_DT.best_score_